In [1]:
def CreateInterface(project, title, titles):
    content = (f'''using {project}.Models;

namespace {project}.Interfaces
{{
    /// <summary>
    /// https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/keywords/interface
    /// An interface defines a contract. Any class or struct that implements that contract
    /// must provide an implementation of the members defined in the interface. An interface
    /// may define a default implementation for members
    /// </summary>
    public interface I{title}Repository
    {{
        ICollection<{title}> Get{titles}();
        {title} Get{title}(int id);

        bool Create{title}({title} {title.lower()});
        bool Update{title}({title} {title.lower()});
        bool Delete{title}({title} {title.lower()});

        bool {title}Exists(int id);
        bool Save();
    }}
}}
/// TODO: Update methods of the I{title}Repository interface, implement interface and create controllers.
/// 1. You have CRUD methods, think about what you want to keep, add, adjust. (maybe you want to get 
/// instances of {title} class by Id of the instance of other class)
/// 2. Implement interface in the {title}Repository and change functions of the methods by your will.
/// 3. Add and adjust controllers in the {title}Controller for every change in the {title}Repository.''')
    return content

In [2]:
def CreateRepository(project, title, titles):
    content = (f'''using {project}.Data;
using {project}.Interfaces;
using {project}.Models;

namespace {project}.Repository
{{
    /// <summary>
    /// https://learn.microsoft.com/en-us/aspnet/mvc/overview/older-versions/getting-started-with-ef-5-using-mvc-4/implementing-the-repository-and-unit-of-work-patterns-in-an-asp-net-mvc-application
    /// The repository and unit of work patterns are intended to create an abstraction layer between the
    /// data access layer and the business logic layer of an application. Implementing these patterns can 
    /// help insulate your application from changes in the data store and can facilitate automated unit 
    /// testing or test-driven development (TDD).
    /// </summary>
    public class {title}Repository : I{title}Repository
    {{
        #region ConstructorAndPrivateElement
        /// <summary>
        /// https://learn.microsoft.com/en-us/dotnet/api/system.data.linq.datacontext?view=netframework-4.8.1
        /// The DataContext is the source of all entities mapped over a database connection.
        /// </summary>
        private DataContext _context;

        public {title}Repository(DataContext context)
        {{
            _context = context;
        }}
        #endregion

        #region GetAllInstances
        /// <summary>
        /// Get all instances oredered by Id.
        /// </summary>
        /// <returns>List of all instances.</returns>
        public ICollection<{title}> Get{titles}()
        {{
            return _context.{titles}.OrderBy(x => x.Id).ToList();
        }}
        #endregion

        #region GetInstanceById
        /// <summary>
        /// Get first or default instance with matching Id searched with Where() function.
        /// </summary>
        /// <param name="id"></param>
        /// <returns>Certain instance with matching Id.</returns>
        public {title} Get{title}(int id)
        {{
            return _context.{titles}.Where(x => x.Id == id).FirstOrDefault();
        }}
        #endregion

        #region CreateInstance
        /// <summary>
        /// Create Instance of {title} class.
        /// </summary>
        /// <param name="{title.lower()}"></param>
        /// <returns>true or false, whether the Save() was successfull or not.</returns>
        public bool Create{title}({title} {title.lower()})
        {{
            _context.Add({title.lower()});
            return Save();
        }}
        #endregion

        #region UpdateInstance
        /// <summary>
        /// Update ceratin instance of {title} class.
        /// </summary>
        /// <param name="{title.lower()}"></param>
        /// <returns>true or false, whether the Save() was successfull or not.</returns>
        public bool Update{title}({title} {title.lower()})
        {{
            _context.Update({title.lower()});
            return Save();
        }}
        #endregion

        #region DeleteInstance
        /// <summary>
        /// Delete certain instance of {title} class.
        /// </summary>
        /// <param name="{title.lower()}"></param>
        /// <returns>true or false, whether the Save() was successfull or not.</returns>
        public bool Delete{title}({title} {title.lower()})
        {{
            _context.Remove({title.lower()});
            return Save();
        }}
        #endregion

        #region CheckIfInstanceExistsById
        /// <summary>
        /// Check if instance exist with selected Id checked with Any() function.
        /// </summary>
        /// <param name="id"></param>
        /// <returns>true or false, whether Any() function found selected Id or not.</returns>
        public bool {title}Exists(int id)
        {{
            return _context.{titles}.Any(x => x.Id == id);
        }}
        #endregion

        #region SaveChanges
        /// <summary>
        /// Saves changes of the _context instance (DataContext class).
        /// </summary>
        /// <returns>true or false, whether SaveChanges() was successfull or not.</returns>
        public bool Save()
        {{
            var saved = _context.SaveChanges();
            return saved > 0 ? true : false;
        }}
        #endregion            
    }}
}}''')
    return content

In [3]:
def CreateController(project, title, titles):
    content = (f'''using AutoMapper;
/// TODO: NuGet packages AutoMapper and AutoMapper.Extensions.Microsoft.DependencyInjection (by Jimmy Bogard)
using Microsoft.AspNetCore.Mvc;
using {project}.Interfaces;
using {project}.Models;

namespace PokemonReviewApp.Controllers
{{
    /// <summary>
    /// https://learn.microsoft.com/en-us/aspnet/mvc/overview/older-versions-1/controllers-and-routing/aspnet-mvc-controllers-overview-cs
    /// MVC controllers are responsible for responding to requests made against an ASP.NET MVC website. 
    /// Each browser request is mapped to a particular controller.
    /// A controller exposes controller actions. 
    /// An action is a method on a controller that gets called when you enter a particular URL in your browser address bar.
    /// </summary>
    [Route("api/[controller]")]
    [ApiController]
    public class {title}Controller : Controller
    {{
        #region Controller and private parts
        /// <summary>
        /// Private: I{title}Repository instance and IMapper instance.
        /// </summary>
        private readonly I{title}Repository _{title.lower()}Repository;
        private readonly IMapper _mapper;

        public {title}Controller(I{title}Repository {title.lower()}Repository, IMapper mapper)
        {{
            _{title.lower()}Repository = {title.lower()}Repository;
            _mapper = mapper;
        }}
        #endregion
        
        #region GetAll
        /// <summary>
        /// Calls Get{titles}() method and mapps output to the List of {title}Dto with help of _mapper instance of the IMapper.
        /// Checks if the model state is valid and returns BadRequest() (400) response if it's not.
        /// </summary>
        /// <returns>Ok() response (200) with the {titles.lower()}, list of mapped data to the {title}Dto.</returns>
        [HttpGet]
        [ProducesResponseType(200, Type = typeof(IEnumerable<{title}>))]
        public IActionResult Get{titles}()
        {{
            var {titles.lower()} = _mapper.Map<List<{title}Dto>>(_{title.lower()}Repository.Get{titles}());

            if (!ModelState.IsValid)
                return BadRequest(ModelState);

            return Ok({titles.lower()});
        }}
        #endregion
        
        #region GetOneById
        /// <summary>
        /// Checks if instance with selected Id exists with {title}Exists function.
        /// Calls Get{title}({title}Id) method and maps output to the {title}Dto with help of _mapper instance of the IMapper.
        /// Checks if the model state is valid and returns BadRequest() (400) response if it's not.
        /// </summary>
        /// <param name="{title}Id"></param>
        /// <returns>Ok() response (200) with the {title.lower()}, mapped data to the {title}Dto.</returns>
        [HttpGet("{{{title.lower()}Id}}")]
        [ProducesResponseType(200, Type = typeof({title}))]
        [ProducesResponseType(400)]
        public IActionResult Get{title}(int {title.lower()}Id)
        {{
            if (!_{title.lower()}Repository.{title}Exists({title.lower()}Id))
                return NotFound();

            var {title} = _mapper.Map<{title}Dto>(_{title.lower()}Repository.Get{title}({title.lower()}Id));
            if (!ModelState.IsValid)
                return BadRequest(ModelState);

            return Ok({title});
        }}
        #endregion
        
        #region Post
        /// <summary>
        /// From body of the POST request it loads {title.lower()}Create, instance of the {title}Dto.
        /// Checks if the instance is null and returns BadRequest() (400) if it is.
        /// Checks by the Name variable if there already exists the same instance.
        /// Change Name to whatever variable you use, or delete that complete part.
        /// Checks if the model state is valid and returns BadRequest() (400) response if it's not.
        /// Mapps data to the {title} model.
        /// Calls Create{title} method and if the method returns false (data vas not saved),
        /// the return response will be 500.
        /// </summary>
        /// <param name="{title.lower()}Create"></param>
        /// <returns>Retrns Ok() response (200), with "Successfully created!" message.<returns>
        [HttpPost]
        [ProducesResponseType(204)]
        [ProducesResponseType(400)]
        public IActionResult Create{title}([FromBody] {title}Dto {title.lower()}Create)
        {{
            if ({title.lower()}Create == null)
                return BadRequest(ModelState);
                
            /// TODO: Change the Name variable (POST, {title})
            /// to whichever variable (or multiple variables) that is implemented in the {title} model,
            /// and is good enough for checking if the same istance of {title} class already exists.
            /// If there is no such variable, delete the following part to avoid errors.
            var {titles.lower()} = _{title.lower()}Repository.Get{titles}()
                .Where(c => c.Name.Trim().ToUpper() == {title.lower()}Create.Name.TrimEnd().ToUpper()).FirstOrDefault();

            if ({titles.lower()} != null)
            {{
                ModelState.AddModelError("", "{title} already exisits");
                return StatusCode(422, ModelState);
            }}

            if (!ModelState.IsValid)
                return BadRequest(ModelState);

            var {title.lower()}Map = _mapper.Map<{title}>({title.lower()}Create);

            if (!_{title.lower()}Repository.Create{title}({title.lower()}Map))
            {{
                ModelState.AddModelError("", "Something went wrong while saving");
                return StatusCode(500, ModelState);
            }}

            return Ok("Successfully created!");
        }}
        #endregion
        
        #region Put
        /// <summary>
        /// Gets {title.lower()}Id from query and updated{title} instance of the {title}Dto from the body of request.
        /// Checks if updated{title} instance is null, and returns BadRequest() (400) if it is.
        /// Checks if the {title.lower()}Id is different than the Id of the updated{title}, and returns BadRequest() (400).
        /// Checks if the {title}Exists() with NotFound() (404) response if not.
        /// Checks if the ModelState is valid, BadRequest() (400) if not.
        /// Maps updated{title} data to the {title} Model.
        /// Calls Update{title} method with response 500 if the Save() produced error.
        /// </summary>
        /// <param name="{title.lower()}Id"></param>
        /// <param name="updated{title}"></param>
        /// <returns>NoContent() response (204).</returns>
        [HttpPut("{{{title.lower()}Id}}")]
        [ProducesResponseType(204)]
        [ProducesResponseType(400)]
        [ProducesResponseType(404)]
        public IActionResult Update{title}(int {title.lower()}Id, [FromBody] {title}Dto updated{title})
        {{
            if (updated{title} == null)
                return BadRequest(ModelState);

            if ({title.lower()}Id != updated{title}.Id)
                return BadRequest(ModelState);

            if (!_{title.lower()}Repository.{title}Exists({title.lower()}Id))
                return NotFound();

            if (!ModelState.IsValid)
                return BadRequest(ModelState);

            var {title.lower()}Map = _mapper.Map<{title}>(updated{title});

            if (!_{title.lower()}Repository.Update{title}({title.lower()}Map))
            {{
                ModelState.AddModelError("", "Something went wrong while updating");
                return StatusCode(500, ModelState);
            }}

            return NoContent();
        }}
        #endregion
        
        #region Delete
        /// <summary>
        /// Gets {title.lower()}Id from request query.
        /// Checks if {title}Exists with selected {title.lower()}Id, NotFound() (404) if not.
        /// Checks if the ModelState is valid, BadRequest() (400) if not.
        /// Calls Delete{title} function, 500 if Save() had error.
        /// </summary>
        /// <param name="{title.lower()}Id"></param>
        /// <returns>Response NoContent() (204).</returns>
        [HttpDelete("{{{title.lower()}Id}}")]
        [ProducesResponseType(204)]
        [ProducesResponseType(400)]
        [ProducesResponseType(404)]
        public IActionResult Delete{title}(int {title.lower()}Id)
        {{
            if (!_{title.lower()}Repository.{title}Exists({title.lower()}Id))
                return NotFound();

            var {title.lower()}ToDelete = _{title.lower()}Repository.Get{title}({title.lower()}Id);

            if (!ModelState.IsValid)
                return BadRequest(ModelState);

            if (!_{title.lower()}Repository.Delete{title}({title.lower()}ToDelete))
            {{
                ModelState.AddModelError("", "Something went wrong while deleting");
                return StatusCode(500, ModelState);
            }}

            return NoContent();
        }}
        #endregion
    }}
}}''')
    return content

In [4]:
def MappingProfiles(model_dict):
    sentence = ''
    for title in model_dict.keys(): 
        sentence += f'''CreateMap<{title}, {title}Dto>();
            '''
    for title in model_dict.keys(): 
        sentence += f'''
            CreateMap<{title}Dto, {title}>();'''
    return sentence

def CreateMappingProfiles(project, model_dict):
    content = (f'''using AutoMapper;
using {project}.Dto;
using {project}.Models;

namespace {project}.Helper
{{
    public class {project} : Profile
    {{
        public MappingProfiles()
        {{
            {MappingProfiles(model_dict)}
        }}
    }}
}}''')
    return content

In [5]:
def DataContextHelper(model_dict):
    sentence = ''
    for title in model_dict.keys(): 
        sentence += f'''public DbSet<{title}> {model_dict[title]} {{ get; set; }}
        '''        
    return sentence

def CreateDataContext(project, model_dict):
    content = (f'''using Microsoft.EntityFrameworkCore;
/// TODO: NuGet package Microsoft.EntityFrameworkCore.SqlServer (by Microsoft)
using {project}.Models;

namespace {project}.Data
{{
    /// <summary>
    /// https://learn.microsoft.com/en-us/dotnet/api/system.data.entity.dbcontext?view=entity-framework-6.2.0
    /// A DbContext instance represents a combination of the Unit Of Work and Repository
    /// patterns such that it can be used to query from a database and group together
    /// changes that will then be written back to the store as a unit. DbContext is
    /// conceptually similar to ObjectContext.
    /// </summary>
    public class DataContext : DbContext
    {{
        /// <summary>
        /// https://learn.microsoft.com/en-us/dotnet/api/system.data.linq.datacontext?view=netframework-4.8.1
        /// The DataContext is the source of all entities mapped over a database connection. It tracks changes
        /// that you made to all retrieved entities and maintains an "identity cache" that guarantees that
        /// entities retrieved more than one time are represented by using the same object instance.
        /// </summary>
        /// <param name="options"></param>
        public DataContext(DbContextOptions<DataContext> options) : base(options)
        {{
        }}
        
        {DataContextHelper(model_dict)}
        protected override void OnModelCreating(ModelBuilder modelBuilder)
        {{
            /// TODO: Create connections between models (DataContext)
            /// 
            /// Example:
            /// modelBuilder.Entity<PokemonCategory>()
            ///     .HasKey(pc => new {{ pc.PokemonId, pc.CategoryId }});
            /// modelBuilder.Entity<PokemonCategory>()
            ///     .HasOne(p => p.Pokemon)
            ///     .WithMany(pc => pc.PokemonCategories)
            ///     .HasForeignKey(p => p.PokemonId);
            /// modelBuilder.Entity<PokemonCategory>()
            ///     .HasOne(p => p.Category)
            ///     .WithMany(pc => pc.PokemonCategories)
            ///     .HasForeignKey(c => c.CategoryId);
        }}
    }}
}}''')
    return content

In [6]:
def GenerateDirectories(parent_dir, directory):
    try:
        os.mkdir(os.path.join(parent_dir, directory))
    except OSError as error:
        print(error)

In [8]:
import os
import shutil

PARENT_DIR = r'C:\Users\Agata\Documents\VS_API\Pokus\Pokus'
PROJECT_NAME = 'Pokus'
MODEL_DICT = {'User':'Users', 'Post':'Posts', 'Country':'Countries'}
DIRECTORIES = ['Interfaces', 'Repository', 'Controllers', 'Dto', 'Data', 'Helper']
DTO_HELPER = ('''
/// <summary>
/// https://learn.microsoft.com/en-us/aspnet/web-api/overview/data/using-web-api-with-entity-framework/part-5
/// A DTO is an object that defines how the data will be sent over the network.
/// </summary>

/// TODO: Change DTO to show only data you want to be shown.''')

print('''There will be changes to multiple files, all previous data will be erased.
Do you want to proceede? y = Yes''')
proceede = input()

if (proceede.lower()=='y'):
    for DIRECTORY in DIRECTORIES:
        GenerateDirectories(PARENT_DIR, DIRECTORY)

    path = os.path.join(PARENT_DIR, 'Data', 'DataContext.cs')
    f = open(path, 'w')
    f.write(CreateDataContext(PROJECT_NAME, MODEL_DICT))
    f.close()

    path = os.path.join(PARENT_DIR, 'Helper', 'MappingProfiles.cs')
    f = open(path, 'w')
    f.write(CreateMappingProfiles(PROJECT_NAME, MODEL_DICT))
    f.close()

    for title in MODEL_DICT.keys():
        src_file = os.path.join(PARENT_DIR, 'Models', f'{title}.cs')
        destination = os.path.join(PARENT_DIR, 'Dto', f'{title}Dto.cs')

        fin = open(src_file, "rt")
        fout = open(destination, "wt")
        for line in fin:
            fout.write(line.replace(f'public class {title}', f'public class {title}Dto'))
        fin.close()
        fout.close()
        f = open(destination, 'a')
        f.write(DTO_HELPER)
        f.close()

        path = os.path.join(PARENT_DIR, 'Interfaces', f'I{title}Repository.cs')
        f = open(path, 'w')
        f.write(CreateInterface(PROJECT_NAME, title, MODEL_DICT[title]))
        f.close()

        path = os.path.join(PARENT_DIR, 'Repository', f'{title}Repository.cs')
        f = open(path, 'w')
        f.write(CreateRepository(PROJECT_NAME, title, MODEL_DICT[title]))
        f.close()

        path = os.path.join(PARENT_DIR, 'Controllers', f'{title}Controller.cs')
        f = open(path, 'w')
        f.write(CreateController(PROJECT_NAME, title, MODEL_DICT[title]))
        f.close()

There will be changes to multiple files, all previous data will be erased.
Do you want to proceede? y = Yes
y
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Agata\\Documents\\VS_API\\Pokus\\Pokus\\Interfaces'
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Agata\\Documents\\VS_API\\Pokus\\Pokus\\Repository'
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Agata\\Documents\\VS_API\\Pokus\\Pokus\\Controllers'
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Agata\\Documents\\VS_API\\Pokus\\Pokus\\Dto'
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Agata\\Documents\\VS_API\\Pokus\\Pokus\\Data'
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Agata\\Documents\\VS_API\\Pokus\\Pokus\\Helper'
